In [2]:
import fitz  # PyMuPDF library (also known as Fitz) for PDF parsing, extracting
import re
import os
import csv
import pandas as pd
import glob

In [91]:
pdf_path = "C:/Users/docli/sad added"
pdf_files = [file for file in os.listdir(pdf_path) if file.endswith('.pdf')]

In [92]:
len(pdf_files)

2

In [93]:
#Load and extract text from the PDF:
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as pdf_document:
        num_pages = pdf_document.page_count
        for page_num in range(num_pages):
            page = pdf_document.load_page(page_num)
            text += page.get_text("text")
    return text

In [94]:
def clean_newlines_extraspaces(text):
    # Remove extra spaces, newlines, and multiple spaces
    cleaned_text = re.sub(r'\s+', ' ', text).strip()
    return cleaned_text

In [95]:
#Extarcting and storing into list called extracted_news
extracted_news=[] #Create an empyt list to store texts from every pdf file in our newsdata folder
for file in pdf_files:
    filePath=open(pdf_path+'/'+file,'r')#locate and open the pdf file and read it
    extracted_news.append(extract_text_from_pdf(filePath)) # pass the text extracting function from pdf and add to our list

In [96]:
def remove_text_before_delimiter(text, delimiter):
    remaining_text = text.split(delimiter)[-1].strip()
    return remaining_text

In [97]:
segmented_news=[]
for news in extracted_news:
    segmented_news.append(remove_text_before_delimiter(news,"\nBody\n"))

In [98]:
#split list elements into two parts when either "Graphic" or "Load-Date:" is found
def split_elements_by_delimiters(input_list, delimiter1, delimiter2):
    amount=0
    new_list = []
    for item in input_list:
        if delimiter1 in item:
            segments = re.split(f"{delimiter1}", item)
            new_list.extend(segments)
        elif delimiter2 in item:
            segments = re.split(f"{delimiter2}", item)
            new_list.extend(segments)
        else:
            new_list.append("News without date")
    return new_list

In [99]:
updated_segmented_news=split_elements_by_delimiters(segmented_news, "Graphic\n" , "Load-Date:")

In [100]:
updated_segmented_news_date = [item for item in updated_segmented_news if item != "News without date"]

In [101]:
def extract_dates_from_list(input_list):
    date_pattern = r"\b(?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2},\s+\d{4}\b"
    dates = [re.search(date_pattern, item).group() for item in input_list if re.search(date_pattern, item)]
    return dates

In [102]:
newsdate=extract_dates_from_list(updated_segmented_news_date[1::2])

In [103]:
len(updated_segmented_news_date[0::2])

2

In [104]:
len(list(newsdate))

2

In [105]:
#replace the elements at odd indices of a list with news date
for i in range(1, len(updated_segmented_news_date), 2):  # Iterate over odd indices
        updated_segmented_news_date[i] = newsdate.pop(0)

In [106]:
for i in range(0, len(updated_segmented_news_date), 2):  # Iterate over even indices
        updated_segmented_news_date[i] = clean_newlines_extraspaces(updated_segmented_news_date[i])

In [107]:
        
def clean_urls_phone_emails(item):
    
    # Remove website URLs
    item = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', item)

    # Remove phone numbers (formats: XXX-XXX-XXXX, (XXX) XXX-XXXX)
    item = re.sub(r'\b(?:\d{3}-\d{3}-\d{4}|\(\d{3}\) \d{3}-\d{4})\b', '', item)

    # Remove emails
    item = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b', '', item)

    # Remove links or placeholders
    item = re.sub(r'\b(?:http[s]?://\S+|www\.\S+|\b\[link\]\b)\b', '', item, flags=re.IGNORECASE)
    
    return item

In [108]:
for i in range(0, len(updated_segmented_news_date), 2):  # Iterate over even indices
    updated_segmented_news_date[i] = clean_urls_phone_emails(updated_segmented_news_date[i])

In [43]:
Anger=updated_segmented_news_date

In [192]:
Anticipation=updated_segmented_news_date

In [88]:
Disgust=updated_segmented_news_date

In [22]:
Fear=updated_segmented_news_date

In [84]:
Joy=updated_segmented_news_date

In [109]:
Sadness=updated_segmented_news_date

In [64]:
Surprise=updated_segmented_news_date

In [172]:
Trust=updated_segmented_news_date

In [112]:
def save_list_to_csv(even_index_values, odd_index_values, filename):
    with open(filename, 'w', newline='',encoding= "utf-8") as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["News Article", "News Date","Sentiment","Remark"])
        
        max_length = max(len(even_index_values), len(odd_index_values))
        
        for i in range(max_length):
            even_value = even_index_values[i] if i < len(even_index_values) else ""
            odd_value = odd_index_values[i] if i < len(odd_index_values) else ""
            csv_writer.writerow([even_value, odd_value,2,'sadness'])

In [322]:
Surprise[5]

'December 18, 2022'

In [323]:
Disgust[5]

'April 28, 2023'

In [113]:
original_list = Sadness
even_index_values = [original_list[i] for i in range(len(original_list)) if i % 2 == 0]
odd_index_values = [original_list[i] for i in range(len(original_list)) if i % 2 != 0]

save_list_to_csv(even_index_values, odd_index_values, "sadness_2.csv")


In [330]:
df_S=pd.read_csv("C:/Users/docli/Surprise.csv")

In [329]:
print(df)

                                         News Article           News Date  \
0   Als 75-Jähriger rüttelt man nicht mehr an pers...   November 20, 2022   
1   Schon bald dürfte dem Dornbirner mehr Zeit für...  September 18, 2022   
2   Mehr als ein Jahr nach Kriegsbeginn in der Ukr...      April 28, 2023   
3   Viktor GröllBad Birnbach Nach zwei Jahren Zwan...    January 12, 2023   
4   Im Gegensatz zu Herrn Merey bin ich sehr froh,...    January 10, 2023   
5   Altbürgermeister Klaus von Dohnanyi wird 95 - ...       June 18, 2023   
6   Von Ansgar Haase, Michel Winde und Andreas Ste...    February 3, 2023   
7   Eine gefährliche Eskalation zeichnet sich ab F...    February 3, 2023   
8   Selbst das Banale wird im Krieg zum Luxus. In ...   November 24, 2022   
9   Jan 04, 2023( ScienceFiles:  Delivered by News...     January 4, 2023   
10  Der Wirtschaftsminister wehrt sich gegen die A...   September 6, 2022   
11  Der Krieg in der Ukraine beschäftigt unsere Le...    February 6, 2023   

In [331]:
print(df_S)

                                         News Article           News Date  \
0   Schönbrunn am Lusen. ,Gemeinsam das Erinnern e...   November 20, 2022   
1   Energie-GAU, Inflation, Rezession, Panik und v...  September 16, 2022   
2   Der Ausfall von 18 Puma-Schützenpanzern bei ei...   December 18, 2022   
3   Der überraschende Besuch von Außenministerin A...  September 11, 2022   
4   Washington - In der Debatte um die Entsendung ...    January 24, 2023   
5   Der Präsident war voll des Lobes für das verbü...   February 27, 2023   
6   Daniel Gerny, Erich Aschwanden Seit Wochen ste...    January 24, 2023   
7   Berlin. Die Bundesregierung ist nun offensicht...    January 24, 2023   
8   25 stillgelegte Kampfpanzer vom Typ Leopard 2 ...        May 24, 2023   
9   Nun also doch: Die Bundesregierung liefert der...    January 24, 2023   
10  Emden/voi - Sein Alltag habe sich in den verga...    February 4, 2023   
11  Bevor die Ukraine die ersten Kampfpanzer aus w...    January 26, 2023   

In [293]:
import glob
import pandas as pd


In [115]:
interesting_files = glob.glob("C:/Users/docli/*.csv") 
df = pd.concat((pd.read_csv(f, header = 0) for f in interesting_files))
df_deduplicated = df.drop_duplicates()
df_deduplicated.to_csv("sentimenstdata_V3.csv")


In [118]:
df=pd.read_csv("C:/Users/docli/sentimentsdata_V3.csv")

In [119]:
df

,Unnamed: 0,News Article,News Date,Sentiment,Remark,Unnamed: 0.1
0,0,Während die Ukraine weiter auf Kampfpanzer aus...,"January 23, 2023",3,Anger,NaN
1,1,"Es kommt selten vor, dass ein General öffentli...","July 30, 2023",3,Anger,NaN
2,2,VON MARKUS DECKER UND SVEN CHRISTIAN SCHULZ (R...,"December 1, 2022",3,Anger,NaN
3,0,Sie können es nicht richtig machen Heft 41/202...,"October 14, 2022",5,Disgust,NaN
4,1,Ein Ausweg aus dem Wolkenkuckucksheim Immer wi...,"February 2, 2023",5,Disgust,NaN
...,...,...,...,...,...,...
751,741,Ramstein. Auch bei dem hochrangig besetzten Tr...,"January 21, 2023",7,Trust,127.0
752,742,Ein Leser teilt seine Ansichten zum Krieg in d...,"January 27, 2023",7,Trust,128.0
753,743,Wir treffen den ukrainischen Botschafter in se...,"June 17, 2023",7,Trust,129.0
754,0,Seit einem Jahr greift Russland die Ukraine an...,"February 19, 2023",6,Surprise,NaN


In [303]:
df['News Article'][2]

'Nov 08, 2022( ScienceFiles:  Delivered by Newstex) David Dunning und Justin Kruger haben ihren eigenen Effekt: mit dem Dunning-Kruger-Effekt üben sie erheblichen Einfluss auf die Erklrung von Leuten aus, die in der Inbrunst eigener, falscher berzeugung den gröten Blödsinn erzhlen und sich dabei wichtig fühlen. Je gröer die Inkompetenz, desto gröer die Klappe, so der Effekt, den Dunning und Kruger in ihren Forschungen entdeckt haben. Und dabei bleiben sie stehen. Denn hier endet ihr Forschungsinteresse. Unseres fngt in gewisser Weise erst an. Wie kann man diese Travestien auf einen ernsthaften Menschen erklren? Was ist in ihrer Sozialisation schief gelaufen? Welches Persönlichkeitssyndrom steckt hinter der Fehlentwicklung zu einem inkompetenten Gromaul? Zumindest auf die letzte Frage haben wir einige Antworten im Verlauf der letzten Monate und Jahre gesammelt. Offenkundig findet sich unter denen, die mit groer Klappe Blödsinn verbreiten, ein nicht geringer Anteil von Leuten, die das in